In [1]:
import cdsapi
import os, sys
from pathlib import Path
import time
from tqdm import tqdm
from loguru import logger
import numpy as np
import pandas as pd
import xarray as xr
import requests
from tqdm import tqdm

# Download CHIRTS files



This notebook goes through the process of downloading [CHIRTS daily Tmin and Tmax daily data](https://iridl.ldeo.columbia.edu/SOURCES/.UCSB/.CHIRTS/.v1.0/.daily/.global/.0p05/index.html?Set-Language=fr) available per year.

### Input parameters

In [2]:
DEST_PATH = Path("../../data/01-raw/chirts")

### Setup logging

In [3]:
LOG_PATH = Path("../../logs/")

# Configure logger
logger.remove()
logger.add(sys.stderr, format="{time} {level} {message}", level="INFO")

# Configure daily rotation for file logging
daily_sink_file_fmt = LOG_PATH / "chirts_{time:YYYY-MM-DD}.log"
logger.add(
    daily_sink_file_fmt,
    rotation="00:00",
    format="{time} {level} {message}",
    level="INFO",
)

2

### Create download function

In [11]:
def download_file(url, save_path):
    file_name = str(save_path).split('/')[-1]
    logger.info(
        "==========================================================================================="
    )
    logger.info(f"Downloading: {file_name}")
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024 * 1024  # 1 Megabyte
    logger.info(f"Total size: {(total_size/ block_size):.2f} MB")
    progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
    with open(save_path, 'wb') as f:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            f.write(data)
    progress_bar.close()
    
    if total_size != 0 and progress_bar.n != total_size:
        logger.error(f"Downloading {file_name}  failed.")
    else:
        logger.success(f"{file_name} successfully downloaded!")

### Download yearly files and subset to PH

In [5]:
PH_BBOX = (116.5, 4.25, 127, 21.5)
years = [2013,2014]

In [25]:
for year in years:
    for data_type in ['min','max']:
        try:
            file_url = f"https://data.chc.ucsb.edu/products/CHIRTSdaily/v1.0/global_netcdf_p05/T{data_type}/T{data_type}.{year}.nc"
            download_file(file_url, DEST_PATH / "tmp"/ f"T{data_type}_{year}.nc")
            # Subset to PH 
            ds = xr.open_dataset(DEST_PATH / "tmp"/ f"T{data_type}_{year}.nc")
            ds = ds.sel(latitude=slice(PH_BBOX[1], PH_BBOX[3]), longitude=slice(PH_BBOX[0], PH_BBOX[2]))
            ds.to_netcdf(DEST_PATH / f"CHIRTS_T{data_type}_PH_{year}.nc")
            os.remove(DEST_PATH / "tmp"/ f"T{data_type}_{year}.nc")
        except KeyboardInterrupt:
            logger.error("Process interrupted using keyboard.")
            break
        

2024-03-26T16:01:05.796690+0800 INFO ===========================================================================================
2024-03-26T16:01:05.798647+0800 INFO Downloading: Tmin_2013.nc
2024-03-26T16:01:07.586977+0800 INFO Total size: 26069.61 MB
  0%|          | 2.10M/27.3G [00:01<4:39:09, 1.63MiB/s]2024-03-26T16:01:09.082865+0800 ERROR Process interrupted using keyboard.
  0%|          | 2.10M/27.3G [00:01<5:24:58, 1.40MiB/s]
2024-03-26T16:01:09.085423+0800 INFO ===========================================================================================
2024-03-26T16:01:09.086233+0800 INFO Downloading: Tmin_2014.nc
2024-03-26T16:01:09.753939+0800 INFO Total size: 26069.61 MB
  1%|          | 166M/27.3G [00:12<35:46, 12.7MiB/s]   